In [1]:
import nltk,os
nltk.download('gutenberg')

from nltk.corpus import gutenberg
import pandas as pd

data=gutenberg.raw('shakespeare-hamlet.txt')

os.makedirs('data',exist_ok=True)

with open('data/hamlet.txt','w') as file:
    file.write(data)

[nltk_data] Downloading package gutenberg to C:\Users\Rishad
[nltk_data]     R\AppData\Roaming\nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!


In [2]:
import numpy as np
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.model_selection import train_test_split

In [3]:
with open('data/hamlet.txt','r') as file:
    text=file.read().lower()
    
tokenizer=Tokenizer()
tokenizer.fit_on_texts([text])
total_words=len(tokenizer.word_index)+1
print(total_words)

4818


In [ ]:
tokenizer.word_index

In [5]:
inputsequences=[]

for line in text.split('\n'):
    token_list=tokenizer.texts_to_sequences([line])[0]
    for i in range(1,len(token_list)):
        n_gram_sequences=token_list[:i+1]
        inputsequences.append(n_gram_sequences)

In [ ]:
inputsequences

In [7]:
max_len=max([len(x) for x in inputsequences])
max_len

14

In [8]:
input_sequences=np.array(pad_sequences(inputsequences,maxlen=max_len,padding='pre'))
input_sequences

array([[   0,    0,    0, ...,    0,    1,  687],
       [   0,    0,    0, ...,    1,  687,    4],
       [   0,    0,    0, ...,  687,    4,   45],
       ...,
       [   0,    0,    0, ...,    4,   45, 1047],
       [   0,    0,    0, ...,   45, 1047,    4],
       [   0,    0,    0, ..., 1047,    4,  193]])

In [9]:
x,y=input_sequences[:,:-1],input_sequences[:,-1]
import tensorflow as tf

y=tf.keras.utils.to_categorical(y,num_classes=total_words)

In [10]:
y

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [11]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

In [12]:
x_train

array([[   0,    0,    0, ...,   46,  723,    2],
       [   0,    0,    0, ...,   54, 3663,  321],
       [   0,    0,    0, ...,    0,    0, 1531],
       ...,
       [   0,    0,    0, ...,    0,    0,   57],
       [   0,    0,    0, ...,  397,  133,  309],
       [   0,    0,    0, ...,   19,   11,    5]])

In [27]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM,Embedding,Dense,Dropout

model=Sequential()
model.add(Embedding(total_words,100,input_length=max_len-1))
model.add(LSTM(150,return_sequences=True))
model.add(Dropout(0.4))
model.add(LSTM(100))
model.add(Dropout(0.4))
model.add(Dense(total_words,activation='softmax'))

In [28]:
from tensorflow.keras.callbacks import EarlyStopping

earlystopping=EarlyStopping(monitor='val_loss',patience=3,restore_best_weights=True)

In [29]:
model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])
model.summary()

Model: "sequential_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_4 (Embedding)     (None, 13, 100)           481800    
                                                                 
 lstm_7 (LSTM)               (None, 13, 150)           150600    
                                                                 
 dropout_4 (Dropout)         (None, 13, 150)           0         
                                                                 
 lstm_8 (LSTM)               (None, 100)               100400    
                                                                 
 dropout_5 (Dropout)         (None, 100)               0         
                                                                 
 dense_3 (Dense)             (None, 4818)              486618    
                                                                 
Total params: 1219418 (4.65 MB)
Trainable params: 1219

In [30]:
history=model.fit(x_train,y_train,
                  epochs=50,
                  validation_data=(x_test,y_test),
                  verbose=1,
                  )

Epoch 1/50
644/644 [==============================] - 24s 31ms/step - loss: 6.8335 - accuracy: 0.0321 - val_loss: 6.6495 - val_accuracy: 0.0359
Epoch 2/50
644/644 [==============================] - 20s 31ms/step - loss: 6.5045 - accuracy: 0.0347 - val_loss: 6.7259 - val_accuracy: 0.0369
Epoch 3/50
644/644 [==============================] - 22s 34ms/step - loss: 6.4095 - accuracy: 0.0389 - val_loss: 6.7965 - val_accuracy: 0.0464
Epoch 4/50
644/644 [==============================] - 23s 35ms/step - loss: 6.3177 - accuracy: 0.0417 - val_loss: 6.8581 - val_accuracy: 0.0499
Epoch 5/50
644/644 [==============================] - 20s 31ms/step - loss: 6.2250 - accuracy: 0.0494 - val_loss: 6.8997 - val_accuracy: 0.0530
Epoch 6/50
644/644 [==============================] - 22s 34ms/step - loss: 6.1357 - accuracy: 0.0526 - val_loss: 6.9380 - val_accuracy: 0.0554
Epoch 7/50
644/644 [==============================] - 20s 31ms/step - loss: 6.0568 - accuracy: 0.0545 - val_loss: 7.0122 - val_accuracy:

In [39]:
def predict_next_word(model,tokenizer,text,max_len):
    token_list=tokenizer.texts_to_sequences([text])[0]
    if len(token_list)>=max_len:
        token_list=token_list[-(max_len-1):]
        
    token_list=pad_sequences([token_list],maxlen=max_len-1,padding='pre')
    predicted=model.predict(token_list,verbose=0)
    predicted_word_index=np.argmax(predicted,axis=1)
    
    for word,index in tokenizer.word_index.items():
        if index==predicted_word_index:
            return word
    return None

In [41]:
input_text="To be or not to be "
print(f'this the input text --{input_text}')
max_sequence_len=model.input_shape[1]+1
next_word=predict_next_word(model,tokenizer,input_text,max_sequence_len)
print(f'next word is --{next_word}')

this the input text --To be or not to be 
next word is --as


In [38]:
model.save('model/model.h5')

e:\machine learning\project\predict next word\venv\lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [36]:
import pickle
with open('model/tokenizer.pkl','wb') as file:
    pickle.dump(tokenizer,file)

